In [6]:
import cv2
import numpy as np
import os
from argparse import ArgumentParser
from tqdm import tqdm
import onnxruntime
import time

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def img2tensor( image):
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    tensor_in = np.expand_dims(image.astype('float32'), 0)
    return tensor_in

def call(img, threshold=0.5):
    tensor = img2tensor(img.copy())
    output = session.run([], {input_name:tensor})[0][0]
    return output


In [4]:
model = "best2.onnx"
size = 384

sess_options = onnxruntime.SessionOptions()
sess_options.enable_profiling = True
session = onnxruntime.InferenceSession(model, providers=['CUDAExecutionProvider'], sess_options=sess_options)
session.get_modelmeta()
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
labels = ['polyp']


In [1]:
video = "Polyp_1.mp4"

cap = cv2.VideoCapture(video)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('Total frame: %d'%num_frames)

output_size = (int(width), int(height))
w, h = 384, 384

if not cap.isOpened():
    print("Can not open video")
    exit(-1)

for i in range(num_frames):
    flag, frame = cap.read()
    if flag:
        start = time.time()
        frame = cv2.resize(frame, (w, h))
        mask = call(frame)
        
        img2 = np.zeros_like(frame)
        img2[:, :, 0] = mask[:, :, 0] * 0
        img2[:, :, 1] = mask[:, :, 1] * 255
        img2[:, :, 2] = mask[:, :, 2] * 255

        #Count FPS
        fps = 1 / (time.time() - start)
        print(start)

        #Draw mask
        result = cv2.bitwise_or(frame, img2)

        #Show prediction
        #im = cv2.hconcat([frame, img2, Result])
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(result, "FPS: " + str(int(fps)), (5, 20), font, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.imshow("Result", result)
        #writer.write(result)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print('Done')

NameError: name 'cv2' is not defined

In [2]:
import onnx
import onnxruntime as ort
import numpy as np
import cv2
import time

MODEL_PATH = 'best2.onnx'
HEIGHT = 384
SAVE_VIDEO = False

model = onnx.load(MODEL_PATH)
onnx.checker.check_model(model)

session = ort.InferenceSession(
    MODEL_PATH,
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

io_binding = session.io_binding()

cap = cv2.VideoCapture('Polyp_1.mp4') # '/home/s/Downloads/Colon.mp4'
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1024) #  1920 1366
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 768) # 1080 768
cv2.namedWindow('polyp', cv2.WINDOW_NORMAL)

if SAVE_VIDEO:
    out_video = cv2.VideoWriter('out_video.avi', cv2.VideoWriter_fourcc(*'XVID'), cap.get(cv2.CAP_PROP_FPS)/3, (740, 768))

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))


while True:
    m1 = time.time()
    success, frame = cap.read()

    frame = frame[:,:740,:]   #crop frame
    input1 = frame.copy()
    input2 = cv2.resize(input1, (HEIGHT, HEIGHT))
    input_ = frame.copy()
    input_ = cv2.resize(input_, (HEIGHT, HEIGHT)) # resize
    input_ = np.float32(input_)    # normalize
    # input_ = input_[:, :, ::-1]   # BRG -> RGB
    # input_ = np.moveaxis(input_, 2, 0)    # HWC -> CHW
    input_ = np.expand_dims(input_, axis=0)
    print(input2.shape)

    m2 = time.time()
    io_binding.bind_cpu_input('data', input_)
    io_binding.bind_output('softmax')
    session.run_with_iobinding(io_binding)
    output = io_binding.copy_outputs_to_cpu()[0]
    output = np.squeeze(output)

    m3 = time.time()
    print(output.shape)
    
    #output = np.argmax(output, axis=0)
    
    img2 = np.zeros_like(input2)
    img2[:, :, 0] = output[:, :, 0] * 0
    img2[:, :, 1] = output[:, :, 1] * 255
    img2[:, :, 2] = output[:, :, 2] * 255
    
    fps = 1 / (time.time() - m1)
    result = cv2.bitwise_or(input2, img2)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, "FPS: " + str(int(fps)), (5,20), font, 0.5, (0,255,0), 1, cv2.LINE_AA)

    cv2.imshow("polyp",result)
    m4 = time.time()
    
    if cv2.waitKey(1) == 27:
        break

    if SAVE_VIDEO:
        out_video.write(frame_copy)

cap.release()
cv2.destroyAllWindows()
        


(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 